In [88]:
from bs4 import BeautifulSoup
from goose3 import Goose
from pprint import pprint
from tqdm import tqdm
import MySQLdb as mdb
import collections
import requests
import json
import re

In [7]:
baca_word = ['Informasi Menarik Terbaru','Membaca:','Baca juga','Baca :','BACA JUGA:','Artikel ini telah tayang di',
            'Baca:','BACA :','Baca Juga:','Baca artikel sumber','Penulis :']

user_agent = ('Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6)'
              ' AppleWebKit/537.36 (KHTML, like Gecko)'
              ' Chrome/58.0.3029.96 Safari/537.36')

known_pattern = json.loads(open("pattern.json", "r").read())

g = Goose({
            'enable_image_fetching': False,
            'use_meta_language': False,
            'target_language': "id",
            'browser_user_agent': user_agent,
            'known_context_patterns': known_pattern,
        })

In [23]:
google_news = [
    {
        "kanal": "bisnis",
        "link": "CAAqJggKIiBDQkFTRWdvSUwyMHZNRGx6TVdZU0FtbGtHZ0pKUkNnQVAB"
    },
    {
        "kanal": "teknologi",
        "link": "CAAqJggKIiBDQkFTRWdvSUwyMHZNRGRqTVhZU0FtbGtHZ0pKUkNnQVAB"
    },
    {
        "kanal": "hiburan",
        "link": "CAAqJggKIiBDQkFTRWdvSUwyMHZNREpxYW5RU0FtbGtHZ0pKUkNnQVAB"
    },
    {
        "kanal": "olahraga",
        "link": "CAAqJggKIiBDQkFTRWdvSUwyMHZNRFp1ZEdvU0FtbGtHZ0pKUkNnQVAB"
    },
    {
        "kanal": "sains",
        "link": "CAAqJggKIiBDQkFTRWdvSUwyMHZNRFp0Y1RjU0FtbGtHZ0pKUkNnQVAB"
    },
    {
        "kanal": "kesehatan",
        "link": "CAAqIQgKIhtDQkFTRGdvSUwyMHZNR3QwTlRFU0FtbGtLQUFQAQ"
    }
]

In [24]:
def get_all_kanal_links():
    all_kanal_links = []
    for kanal in google_news:
        url = "https://news.google.com/topics/{link}?hl=id&gl=ID&ceid=ID%3Aid".format(link=kanal['link'])
        all_kanal_links.append([kanal['kanal'],url])
        
    return all_kanal_links

In [18]:
def cleaning_title(title):
    sparator = [' |',' -',' –']
    for s in sparator:
        title_ = title.split(s)
        
        if title_[0] == title:
            pass
        else:
            if len(title[0]) < 20:
                title_fix = title_[1]
            else:
                title_fix = title_[0]
               
    return title_fix

In [10]:
def cleaning_content(text):    
    junk_word = []
    for word in baca_word:
        for i in text.split('.'):
            if i.find(word) >= 0 :
                junk_word.append(i)

    for j in junk_word:
        text = text.replace(j, ' ')
        
    text = text.replace('..','.').replace('. . ','. ')
        
    return text

In [82]:
def get_content(link):
    redirect = requests.get(link, timeout=5, allow_redirects=True)
    
    if 'www.youtube.com' in redirect.url:
        pass
    else:
        try:
            extract = g.extract(redirect.url)

            title = extract.title
            title = cleaning_title(title)
            final_title = re.sub(r'[^\x00-\x7F]+', '', title)

            content = cleaning_content(extract.cleaned_text)
            content = content.replace('‘','').replace('\n\n','\n').replace('\n\n','\n').replace('\n',' ').replace('  ',' ').replace('- ','')
            content = content.strip()
            final_content = re.sub(r'[^\x00-\x7F]+', '', content)

            if len(content) > 100:
                final_content = final_content
            else:
                final_content = "Not Valid"
            
            if final_title == "" or final_content == "Not Valid" or final_content == "":
                result = "Invalid"
            else:
                result = {
                    'url': redirect.url,
                    'title': final_title,
                    'spoiler': '{content}...'.format(content=final_content[:297]),
                    'content': final_content
                }
        except:
            result = "Invalid"

    return(result)

In [83]:
def get_link_one_kanal(kanal):
    all_kanal_links = get_all_kanal_links()
    link_kanal = declare_link_kanal(kanal)
    page = requests.get(link_kanal)
    soup = BeautifulSoup(page.content, 'html.parser')
    href = soup.find_all("a", {"class" : "VDXfz"})

    links_one_kanal = []
    for h in href:
        h = h['href'].replace('./articles','https://news.google.com/articles')
        links_one_kanal.append(h)
    
    

    return links_one_kanal

In [92]:
def save_to_db(kanal):
    for k in kanal:
        print("Scraping kanal '%s' .." %k)
        links_one_kanal = get_link_one_kanal(k)
        
        con = mdb.connect(host, user, password, db, charset='utf8')
#         counter = 1
        total = len(links_one_kanal)
    
        counter = collections.Counter()
        
        for i in tqdm(range(len(links_one_kanal[:3]))):
            result = get_content(links_one_kanal[i])

            if result == "Invalid":
                counter['Failed'] += 1
#                 print("%d/%d | ERROR | %s " %(counter,total,link))
#                 counter += 1
                pass
            else:
                url = result['url']
                title = result['title']
                spoiler = result['spoiler']
                content = result['content']

                sql = '''INSERT INTO {table} VALUES (0,"{url}","{title}","{spoiler}","{content}",NOW())'''.format(
                    table=k.lower(),
                    url=url,
                    title=title,
                    spoiler=spoiler,
                    content=content
                )

                try:
                    cur = con.cursor()
                    cur.execute(sql)
                    con.commit()
                    
                    counter['Succes'] += 1
#                     print("%d/%d | SUCCES | %s " %(counter,total,title))
#                     counter += 1
                except:
                    counter['Failed'] += 1
#                     print("%d/%d | FAILED | %s " %(counter,total,title))
#                     counter += 1
                    pass
        print(counter)
        print()

In [93]:
all_kanal = ["BISNIS","TEKNOLOGI","HIBURAN","OLAHRAGA","SAINS","KESEHATAN"]

save_to_db(all_kanal)

Scraping kanal 'BISNIS' ..


100%|██████████| 3/3 [00:06<00:00,  2.10s/it]


Counter({'Failed': 3})
Scraping kanal 'TEKNOLOGI' ..


100%|██████████| 3/3 [00:04<00:00,  1.64s/it]


Counter({'Failed': 2, 'Succes': 1})
Scraping kanal 'HIBURAN' ..


...

Counter({'Failed': 2, 'Succes': 1})
Scraping kanal 'SAINS' ..


100%|██████████| 3/3 [00:03<00:00,  1.02s/it]


Counter({'Failed': 3})
Scraping kanal 'KESEHATAN' ..


100%|██████████| 3/3 [00:04<00:00,  1.60s/it]

Counter({'Failed': 3})
